<a href="https://colab.research.google.com/github/SAIKIRANDHULLA/Assignment3-LLM/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessary libraries
from google.colab import drive
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
import torch
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok


In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
train_path = '/content/drive/MyDrive/BERT2/train.csv'
test_path = '/content/drive/MyDrive/BERT2/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
train_df.head()

,class_index,review_text
0,4,well-done film
1,1,characters ramble
2,3,the intensity that made her an interesting cha...
3,1,'s not really funny
4,2,stuffs


In [ ]:
test_df.head()

,class_index,review_text
0,2,at the right time in the history of our country
1,3,of the best silly horror movies
2,2,of tragedies
3,2,a glass
4,2,the fly -- like between lunch breaks for Shear...


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['review_text'],
                      padding='max_length',
                      truncation=True,
                      labels=examples['class_index'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['review_text'], padding='max_length', truncation=True)


In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True)


Map:   0%|          | 0/100693 [00:00<?, ? examples/s]

Map:   0%|          | 0/25174 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.map(lambda examples: {'labels': examples['class_index']}, batched=True)


Map:   0%|          | 0/100693 [00:00<?, ? examples/s]

Map:   0%|          | 0/25174 [00:00<?, ? examples/s]

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead.")


GPU is available and being used.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,0.716100,0.698923
2,0.579300,0.705194


TrainOutput(global_step=12588, training_loss=0.6747122386693576, metrics={'train_runtime': 5825.7093, 'train_samples_per_second': 34.568, 'train_steps_per_second': 2.161, 'total_flos': 5.29883102382551e+16, 'train_loss': 0.6747122386693576, 'epoch': 2.0})

In [ ]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('./saved_model')
tokenizer = AutoTokenizer.from_pretrained('./saved_model')


In [ ]:
# Define a prediction function
def predict_sentiment(text):
    # Tokenize and encode the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)

    # Move tensors to the GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the prediction
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Map the predicted class to sentiment label
    labels = ['very negative', 'negative', 'neutral', 'positive', 'very positive']  # 5 labels
    sentiment = labels[predicted_class]

    return sentiment

# Example usage
test_text = "The movie at the show was absolutely amazing!"
print(f"Sentiment: {predict_sentiment(test_text)}")


Sentiment: very positive


In [ ]:
test_texts = [
    "I didn't enjoy the movie at all; it was a complete waste of time.",
    "The film was okay, but I wasn't particularly impressed or disappointed.",
    "The acting was superb, and the storyline was engaging from start to finish.",
    "It was a decent watch, but I've seen better films in this genre.",
    "I found the movie to be incredibly boring and not worth watching.",
    "The special effects were stunning, but the plot was rather predictable.",
    "I loved every moment of the film; it was a truly captivating experience!",
    "The movie was average; it had its highs and lows but nothing exceptional.",
    "The film exceeded my expectations with its excellent direction and performances.",
    "The storyline was convoluted and difficult to follow, which detracted from the experience.",
    "The movie had a great premise but fell short in execution.",
    "I was thoroughly entertained and would recommend this film to others.",
    "The characters were poorly developed, and the pacing was too slow.",
    "The movie was an emotional rollercoaster and left a lasting impression on me.",
    "I didn't connect with the film at all; it felt like a missed opportunity.",
    "The soundtrack was fantastic, but the rest of the movie was underwhelming.",
    "The film was a delightful surprise; I didn't expect to enjoy it as much as I did.",
    "It was a disappointing sequel that didn't live up to the original.",
    "The movie had its moments, but overall it was forgettable."
]

# Example usage of the function with multiple test texts
for text in test_texts:
    print(f"Text: {text}")
    print(f"Sentiment: {predict_sentiment(text)}")
    print("\n")


Text: I didn't enjoy the movie at all; it was a complete waste of time.
Sentiment: very negative


Text: The film was okay, but I wasn't particularly impressed or disappointed.
Sentiment: positive


Text: The acting was superb, and the storyline was engaging from start to finish.
Sentiment: very positive


Text: It was a decent watch, but I've seen better films in this genre.
Sentiment: positive


Text: I found the movie to be incredibly boring and not worth watching.
Sentiment: very negative


Text: The special effects were stunning, but the plot was rather predictable.
Sentiment: neutral


Text: I loved every moment of the film; it was a truly captivating experience!
Sentiment: very positive


Text: The movie was average; it had its highs and lows but nothing exceptional.
Sentiment: negative


Text: The film exceeded my expectations with its excellent direction and performances.
Sentiment: very positive


Text: The storyline was convoluted and difficult to follow, which detracted fro

In [ ]:
!pip install pyngrok


In [ ]:
!ngrok authtoken YOUR_AUTHTOKEN_HERE


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Initialize the Flask app
app = Flask(__name__)

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('./saved_model')
tokenizer = AutoTokenizer.from_pretrained('./saved_model')

# Define a prediction function
def predict_sentiment(text):
    # Tokenize and encode the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)

    # Move tensors to the GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the prediction
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Assuming the model was trained with these 5 classes
    labels = ['very negative', 'negative', 'neutral', 'positive', 'very positive']
    sentiment = labels[predicted_class]

    return sentiment

# Define the homepage route
@app.route('/')
def home():
    return '''
    <form action="/predict" method="post">
        <h1>Enter Movie Review:</h1>
        <textarea name="review_text" rows="4" cols="50"></textarea><br><br>
        <input type="submit" value="Predict Sentiment">
    </form>
    '''

# Define the prediction route
@app.route('/predict', methods=['POST'])
def predict():
    review_text = request.form['review_text']
    sentiment = predict_sentiment(review_text)
    return render_template_string('''
        <h1>Review: {{ review_text }}</h1>
        <h2>Predicted Sentiment: {{ sentiment }}</h2>
        <a href="/">Go Back</a>
    ''', review_text=review_text, sentiment=sentiment)

# Start ngrok to tunnel the local server
url = ngrok.connect(5000)
print(f"ngrok tunnel available at: {url}")

# Run the app
app.run(host='0.0.0.0', port=5000)


ngrok tunnel available at: NgrokTunnel: "https://6cb3-34-87-27-128.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Aug/2024 19:35:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2024 19:35:52] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2024 19:36:08] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2024 19:36:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2024 19:36:24] "POST /predict HTTP/1.1" 200 -
